In [1]:
import numpy as np 
import matplotlib.pyplot as plt 

In [3]:
def compute_intersections(p0, d):
    # NOTE(cmo): Units of Rs
    a = 1.0
    b = 2.0 * (p0 @ d)
    c = p0 @ p0 - 1.0
    delta = b*b - 4.0 * a * c

    if delta < 0:
        return None
    t1 = (-b - np.sqrt(delta)) / (2.0 * a)
    t2 = (-b + np.sqrt(delta)) / (2.0 * a)
    return t1, t2

def sphere_trace_ctx(ctx, mu, num_sample_points=20_000):
    
    # This defines the viewpoint and the direction
    p0 = np.array([0.0, 0.0, 1.0])
    d = np.array([-np.sqrt(1.0 - mu**2), 0.0, -mu])
    
    # Find the intersections and interpolate there with a fixed number of sub-samples
    # Consider having a fixed lenght of the sample
    intersections = compute_intersections(p0, d)
    if intersections is None:
        raise ValueError("Pass a sane mu pls")
    t1, t2 = intersections
    sample_ts = np.linspace(t1, t2, num_sample_points)
    
    # convert that into sample points in 3D space
    sample_points = p0 + d[None, :] * sample_ts[:, None]
    
    # Now we related these x,y,z to a 1D spherical geometry (here we would want a 3D dependence)
    centre = np.array([0.0, 0.0, 0.0])
    radius = 1.0
    depth = radius - np.sqrt(np.sum((sample_points - centre)**2, axis=1))
    # NOTE(cmo): Dimensionalise and invert sign of depth
    depth *= -const.R_sun.value
    # NOTE(cmo): Shift to align 0 with top of FALC
    depth += ctx.atmos.z[0]
    # NOTE(cmo): Limit depth to maximum of FALC
    mask = depth < ctx.atmos.z[-1]
    if np.any(mask):
        max_depth_index = np.argmax(mask)
        depth[max_depth_index:] = ctx.atmos.z[-1]

    ds = (sample_ts[1] - sample_ts[0]) * const.R_sun.value
    print(f"Step: {ds/1e3} km, max depth = {depth.min() / 1e3} km")

    eta = np.zeros((ctx.spect.wavelength.shape[0], num_sample_points))
    chi = np.zeros((ctx.spect.wavelength.shape[0], num_sample_points))
    
    # Interpolate emissivity and opacity to the ray we are considering. 
    # Here we are starting from a converged falc pp from lw
    for la in range(ctx.spect.wavelength.shape[0]):
        # eta[la, :] = weno4(depth, ctx.atmos.z, ctx.depthData.eta[la, 0, 0, :])
        # chi[la, :] = weno4(depth, ctx.atmos.z, ctx.depthData.chi[la, 0, 0, :])
        eta[la, :] = np.interp(depth, ctx.atmos.z[::-1], ctx.depthData.eta[la, 0, 0, ::-1])
        chi[la, :] = np.interp(depth, ctx.atmos.z[::-1], ctx.depthData.chi[la, 0, 0, ::-1])
    
    # Below is 0th order SC Formal Solver:
    # numerical integration for tau
    dtau = chi * ds
    # dtau = 0.5 * (chi[1:] + chi[:-1]) * ds
    tau = np.cumsum(dtau, axis=1) - tau[0]
    Sfn = eta / chi
    transmission = np.exp(-tau)
    local_contribution = (1.0 - np.exp(-dtau)) * Sfn
    outgoing_contribution = local_contribution * transmission
    I = np.sum(outgoing_contribution, axis=1)
    return I, outgoing_contribution, ds

mu_grid = np.linspace(0.01, 0.1, 19)

In [4]:
p0 = np.array([0,0,2.0])
mu = 1.0
d =  np.array([-np.sqrt(1.0 - mu**2), 0.0, -mu])

print(compute_intersections(p0,d))

(1.0, 3.0)


In [ ]:
# What's gona happen here is that I am gonna keep mu fixed but I am gonna change p0
# for different viewing geometries. So if I want to get, say, CLV of something in,
# spherical gometry. I just need to define my, say, observation direction w.r.t. limb, I say that "limb"
# is at Rs.

In [ ]:
# Let's try multiple p0

for i in range